In [1]:
from utils.seq_evol_embedder import embedder, getSeqEvoLmEmbeddings

##Input
batch = ["PROTEIN","SEQWENCE"] #Takes a list of strings/sequences or just one string/sequence
#batch = "PROTEIN" 

##Generate embeddings
#Returns a list of pytorch tensors embedding the sequences of length (3,L,512)
embeddings = getSeqEvoLmEmbeddings(embedder,batch)

In [2]:
len(embeddings)

2

In [3]:
embeddings[0].shape

torch.Size([3, 7, 512])

In [4]:
embeddings[0][0]

tensor([[-0.0364,  0.0250,  0.0143,  ...,  0.0043,  0.0093, -0.0512],
        [-0.0060, -0.0094,  0.0365,  ..., -0.0537, -0.0663, -0.1020],
        [ 1.0347,  1.0854,  0.6245,  ...,  0.3899,  0.1666, -0.2796],
        ...,
        [-0.0241, -0.0018,  0.0077,  ..., -0.1280, -0.0244, -0.0551],
        [ 0.0357,  0.0893,  0.0157,  ...,  0.0049, -0.0119,  0.0118],
        [-0.0050,  0.0465,  0.0361,  ...,  0.0159, -0.0764, -0.0935]],
       grad_fn=<SelectBackward>)